In [1]:
import importlib

modules_to_check = ['pandas', 'xlrd']

for module in modules_to_check:
    try:
        importlib.import_module(module)
    except ImportError:
        print(f"Module {module} not found. Please install it before running this script.")
        exit(1)

In [2]:
import pandas as pd
import json
import os
import re

In [3]:
excel_file_path = './dlcas-2024_1.2.0_datadictionary_20240606_091810.xls'  # Replace with your file path
# df = pd.read_excel(excel_file_path)
df = pd.read_excel(excel_file_path, sheet_name="comorbiditeiten")
df.head()

,CATEGORIE,LABEL / OMSCHRIJVING,VARIABELE NAAM,TYPE,LENGTE,FORMAAT,VARIABELE VERPLICHT,OPTIE KEUZE VERPLICHT,OPTIESET,STANDAARD WAARDE,ZICHTBAAR,CONDITIE,CALCULATIE,HELP TEKST
0,NaN,SECTIE,Comorbiditeiten 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Algemeen,Datum comorbiditeiten\n\nDit betreft de datum ...,datcom,datum,10.0,jjjj-mm-dd,Ja,NaN,NaN,NaN,NaN,NaN,NaN,Een actuele datum is essentieel voor het koppe...
2,Pulmonaal,Chronische longziekte,comlong,geheel getal,1.0,NaN,NaN,Ja,524.0,"if([../comcopy]==1, plugin('FindLastCreatedVal...",NaN,NaN,NaN,Chronische longziekte (ICD-10):\n\nI27.8: Over...
3,Pulmonaal,COPD/ CARA/ emfyseem/ chron. bronch.,compulmobstr,geheel getal,1.0,NaN,NaN,Ja,524.0,"if([../comcopy]==1, plugin('FindLastCreatedVal...",NaN,NaN,NaN,COPD/CARA/emfyseem/chron.bronch. (ICD-10):\n\n...
4,Pulmonaal,Longfibrose,compulmfibr,geheel getal,1.0,NaN,NaN,Ja,524.0,"if([../comcopy]==1, plugin('FindLastCreatedVal...",NaN,NaN,NaN,Longfibrose (ICD-10):\n\nJ84.1: Overige inters...


In [4]:
# Drop the columns that are not needed
columns_to_delete = [
    'OPTIESET',
    'STANDAARD WAARDE',
    'ZICHTBAAR',
    'CONDITIE',
    'CALCULATIE'
]
df = df.drop(columns=columns_to_delete)

df = df.fillna('')

# Drop rows that have empty strings or are just whitespace in all columns
df = df.dropna(how='all', axis=0)
df = df[~df.apply(lambda x: x.str.strip().eq('')).all(axis=1)]

df.head()

,CATEGORIE,LABEL / OMSCHRIJVING,VARIABELE NAAM,TYPE,LENGTE,FORMAAT,VARIABELE VERPLICHT,OPTIE KEUZE VERPLICHT,HELP TEKST
0,,SECTIE,Comorbiditeiten 1,,,,,,
1,Algemeen,Datum comorbiditeiten\n\nDit betreft de datum ...,datcom,datum,10.0,jjjj-mm-dd,Ja,,Een actuele datum is essentieel voor het koppe...
2,Pulmonaal,Chronische longziekte,comlong,geheel getal,1.0,,,Ja,Chronische longziekte (ICD-10):\n\nI27.8: Over...
3,Pulmonaal,COPD/ CARA/ emfyseem/ chron. bronch.,compulmobstr,geheel getal,1.0,,,Ja,COPD/CARA/emfyseem/chron.bronch. (ICD-10):\n\n...
4,Pulmonaal,Longfibrose,compulmfibr,geheel getal,1.0,,,Ja,Longfibrose (ICD-10):\n\nJ84.1: Overige inters...


In [5]:
df.shape

(37, 9)

In [6]:
# Ensure columns are valid json keys via parsing.
# Parsing rules for the column names:
# - convert to pascal case
# - remove spaces
# - remove special characters
special_chars_regex = r'[^a-zA-Z0-9\s]'

parsed_col_names = [
    re.sub(special_chars_regex, ' ', col_name).title().replace(' ', '')
    for col_name in df.columns
]

df.columns = parsed_col_names
df.columns

Index(['Categorie', 'LabelOmschrijving', 'VariabeleNaam', 'Type', 'Lengte',
       'Formaat', 'VariabeleVerplicht', 'OptieKeuzeVerplicht', 'HelpTekst'],
      dtype='object')

In [7]:
# We populate the sections list so that it can be used to create the JSON
sections = []
current_section = None

for index, data in df.iterrows():
    # If the row is a section, we create a new section JSON object
    if data["LabelOmschrijving"] == "SECTIE":
        section_json = {
            "SectionName": "",
            "SectionItems": []
        }
        section_json["SectionName"] = data["VariabeleNaam"]
        sections.append(section_json)
        current_section = data["VariabeleNaam"]

    # If the row is a section item, we create a new section item JSON object
    else:
        section_item_json = dict.fromkeys(df.columns)
        
        # Populate the section item dictionary with the data from the current row
        for key in parsed_col_names:
            section_item_json[key] = data[key]

        sections[-1]["SectionItems"].append(section_item_json)

In [8]:
print(
    len(sections[0]['SectionItems']) == 24, # True
    len(sections[1]['SectionItems']) == 11  # True
) # Sanity checks

True True


In [9]:
fpath = os.path.abspath(os.path.join(os.getcwd(), 'dlca-s.json'))

if os.path.exists(fpath):
    print(f"Removing existing JSON file at {fpath}")
    os.remove(fpath)

with open(fpath, 'w') as f:
    json.dump(sections, f, indent=4)

print(f"JSON file saved to {fpath}")

Removing existing JSON file at d:\Users\luc\repos\LVDE-OL-MNER-FRONTEND\src\app\stubs\dlca-s.json
JSON file saved to d:\Users\luc\repos\LVDE-OL-MNER-FRONTEND\src\app\stubs\dlca-s.json
